# Creating Visualisations with Dash

Dash is a python library that allows users to create dashboards that can be run in a browser. The dataset being used here is a snippet of energy usage in London Homes. LCLid represents a unique smart meter within a home. This data was collected as part of a project to analyse how different energy tariffs affected home usage across a range of ACORN households. Full information can be found here https://www.kaggle.com/jeanmidev/smart-meters-in-london

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

### Data manipulation

In [2]:
df1 = pd.read_csv('block_1.csv')
df1[['day','time']] = df1['tstp'].str.split(' ', expand = True)
df1.drop(columns = 'tstp', inplace = True)

In [3]:
df1['energy(kWh/hh)'] = pd.to_numeric(df1['energy(kWh/hh)'], errors='coerce')

In [4]:
df1[['year','month','date']] = df1.day.str.split('-', expand = True)
df1

,LCLid,energy(kWh/hh),day,time,year,month,date
0,MAC000323,0.488,2012-03-06,14:00:00.0000000,2012,03,06
1,MAC000323,0.449,2012-03-06,14:30:00.0000000,2012,03,06
2,MAC000323,0.424,2012-03-06,15:00:00.0000000,2012,03,06
3,MAC000323,0.439,2012-03-06,15:30:00.0000000,2012,03,06
4,MAC000323,0.291,2012-03-06,16:00:00.0000000,2012,03,06
...,...,...,...,...,...,...,...
1515860,MAC004529,0.213,2014-02-27,22:00:00.0000000,2014,02,27
1515861,MAC004529,0.227,2014-02-27,22:30:00.0000000,2014,02,27
1515862,MAC004529,0.229,2014-02-27,23:00:00.0000000,2014,02,27
1515863,MAC004529,0.220,2014-02-27,23:30:00.0000000,2014,02,27


In [5]:
df1['month_date'] = df1['month']+'-'+ df1['date']
df1['energy(kWh/hh)'] = pd.to_numeric(df1['energy(kWh/hh)'], errors='coerce')
df1

,LCLid,energy(kWh/hh),day,time,year,month,date,month_date
0,MAC000323,0.488,2012-03-06,14:00:00.0000000,2012,03,06,03-06
1,MAC000323,0.449,2012-03-06,14:30:00.0000000,2012,03,06,03-06
2,MAC000323,0.424,2012-03-06,15:00:00.0000000,2012,03,06,03-06
3,MAC000323,0.439,2012-03-06,15:30:00.0000000,2012,03,06,03-06
4,MAC000323,0.291,2012-03-06,16:00:00.0000000,2012,03,06,03-06
...,...,...,...,...,...,...,...,...
1515860,MAC004529,0.213,2014-02-27,22:00:00.0000000,2014,02,27,02-27
1515861,MAC004529,0.227,2014-02-27,22:30:00.0000000,2014,02,27,02-27
1515862,MAC004529,0.229,2014-02-27,23:00:00.0000000,2014,02,27,02-27
1515863,MAC004529,0.220,2014-02-27,23:30:00.0000000,2014,02,27,02-27


In [17]:
df2 = df1.groupby([df1['year'], df1['day'], df1['LCLid']])
df2

In [18]:
daily_average = df2['energy(kWh/hh)'].mean()

In [19]:
daily_average

year  day         LCLid    
2011  2011-12-21  MAC004529    0.213862
      2011-12-22  MAC004529    0.125333
      2011-12-23  MAC004529    0.197875
      2011-12-24  MAC004529    0.228938
      2011-12-25  MAC004529    0.089687
                                 ...   
2014  2014-02-28  MAC003166    0.250000
                  MAC003182    0.706000
                  MAC003212    0.318000
                  MAC003840    0.146000
                  MAC004529    0.239000
Name: energy(kWh/hh), Length: 31731, dtype: float64

In [9]:
daily_average_df = daily_average.reset_index()
daily_average_df['year'] = daily_average_df['year'].astype('int64')

In [10]:
daily_average_df.dtypes

year                int64
day                object
LCLid              object
energy(kWh/hh)    float64
dtype: object

### Plotting data

### Scatter with a slider

In [13]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=daily_average_df['year'].min(),
        max=daily_average_df['year'].max(),
        value=daily_average_df['year'].min(),
        marks={str(year): str(year) for year in daily_average_df['year'].unique()},
        step=None
    ),
    dcc.Dropdown(id='dropdown')
    options=[
        daily_average_df['LCLid'].unique()
    ],
    value=options[0],
    multi=True

])


@app.callback(
    Output('graph-with-slider', 'figure'),
    [Input('year-slider', 'value')])
def update_figure(selected_year):
    filtered_df = daily_average_df[daily_average_df.year == selected_year]

    fig = px.scatter(filtered_df, x="day", y="energy(kWh/hh)", 
                     color="LCLid", hover_name="LCLid")

    fig.update_layout(transition_duration=500)

    return fig

app.run_server(mode = 'inline')

In [20]:
df1

,LCLid,energy(kWh/hh),day,time,year,month,date,month_date
0,MAC000323,0.488,2012-03-06,14:00:00.0000000,2012,03,06,03-06
1,MAC000323,0.449,2012-03-06,14:30:00.0000000,2012,03,06,03-06
2,MAC000323,0.424,2012-03-06,15:00:00.0000000,2012,03,06,03-06
3,MAC000323,0.439,2012-03-06,15:30:00.0000000,2012,03,06,03-06
4,MAC000323,0.291,2012-03-06,16:00:00.0000000,2012,03,06,03-06
...,...,...,...,...,...,...,...,...
1515860,MAC004529,0.213,2014-02-27,22:00:00.0000000,2014,02,27,02-27
1515861,MAC004529,0.227,2014-02-27,22:30:00.0000000,2014,02,27,02-27
1515862,MAC004529,0.229,2014-02-27,23:00:00.0000000,2014,02,27,02-27
1515863,MAC004529,0.220,2014-02-27,23:30:00.0000000,2014,02,27,02-27
